## ASR

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline


device = "cuda:0"

model_id = "/home/vapa/projects/iskra/models/asr/whisper-large-v3"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch.float16, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch.float16,
    device=device,
)

In [ ]:
sample = "/home/vapa/projects/iskra/received_audio/test.wav"

result = pipe(sample)
print(result["text"])

## LLM

In [5]:
import requests

response = requests.get("http://0.0.0.0:8000/v1/models")
available_models = response.json()
available_models

{'object': 'list',
 'data': [{'id': '/home/vapa/projects/iskra/models/llms/vikhr-12b/',
   'object': 'model',
   'created': 1730804184,
   'owned_by': 'vllm',
   'root': '/home/vapa/projects/iskra/models/llms/vikhr-12b/',
   'parent': None,
   'max_model_len': 4000,
   'permission': [{'id': 'modelperm-5a002d3547c940ffa0f346418963ed73',
     'object': 'model_permission',
     'created': 1730804184,
     'allow_create_engine': False,
     'allow_sampling': True,
     'allow_logprobs': True,
     'allow_search_indices': False,
     'allow_view': True,
     'allow_fine_tuning': False,
     'organization': '*',
     'group': None,
     'is_blocking': False}]}]}

In [7]:
import requests

payload = {
        "model": "/home/vapa/projects/iskra/models/llms/vikhr-12b/",
        "messages": [
            {"role": "system", "content": "Ты виртуальный помощник, который помогает пользователю с его проблемами и отвечает на вопросы."},
            {"role": "user", "content": "Привет, как дела?"}
        ]
    }

response = requests.post("http://0.0.0.0:8000/v1/chat/completions", json=payload)
generated_text = response.json()["choices"][0]["message"]["content"]
generated_text   

'Привет! У меня всё отлично, спасибо, что спросили. Я здесь, чтобы помочь вам. Как я могу помочь вам сегодня?'

# TTS

In [47]:
import requests

url = "http://localhost:5002/api/tts"
headers = {
    "text": "Hello, world you freak man sand",
}

response = requests.post(url, headers=headers)

with open("/home/vapa/projects/iskra/data/tts/responses/output.wav", "wb") as f:
    f.write(response.content)